# Cars: Getting Started

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import pyblp
sns.set_theme()
import matplotlib.pyplot as plt

pyblp.options.digits = 2
pyblp.options.verbose = False

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [2]:
cars = pd.read_csv('cars.csv') # this reads the *balanced* dataset (i.e. J = 40 products per market always)
# cars = pd.read_excel('cars.xlsx') # this reads the *unbalanced* dataset (i.e. J varies over time)

### No data for France pre 1990. Average growth in adult fraction from other countries applied each year before

In [3]:
AdultFrac = pd.read_excel("FracOver20.xlsx", index_col = 0)
cars['adults'] = None
for idx in cars.index:
    cars['adults'][idx] = AdultFrac[cars['ma'][idx]][cars['ye'][idx]]

C:\Users\PSand\AppData\Local\Temp\ipykernel_3308\553329577.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars['adults'][idx] = AdultFrac[cars['ma'][idx]][cars['ye'][idx]]


We estimate that 77% of the adult population have a driving license for a full-car. Hence, the share of population aged 20+ with a driver becomes:

In [4]:
license_share = 0.77
cars["ad_w_li"] = cars["adults"] * license_share 

In [5]:
lbl_vars = pd.read_csv('labels_variables.csv', index_col=0)
lbl_vals = pd.read_stata('cars.dta', iterator=True).value_labels() # the values that variables take (not relevant for all )

## Overview of the dataset

In [6]:
pd.set_option('display.max_colwidth', None)
tab = cars.mean(numeric_only=True).apply(lambda x: f'{x:.2f}').to_frame('Mean').join(lbl_vars)
tab

,Mean,label
ye,84.50,year (=first dimension of panel)
ma,3.00,market (=second dimension of panel)
co,207.50,model code (=third dimension of panel)
zcode,177.76,alternative model code (predecessors and successors get same number)
brd,16.79,brand code
org,2.72,"origin code (demand side, country with which consumers associate model)"
loc,5.17,"location code (production side, country where producer produce model)"
cla,2.30,class or segment code
home,0.32,domestic car dummy (appropriate interaction of org and ma)
frm,14.50,firm code


# Set up for analysis

## Price variables 

Can be either price (`pr`), price-to-income (`princ`), or log price (`logp`, created below).

In [7]:
price_var = 'eurpr'

In [8]:
cars['logp'] = np.log(cars[price_var])

## Market share

**Todo:** Decide how to measure the market size and thereby the market share. *Note:* Below is just an example that sets the market size = population / 3. 

In [9]:
# total quantity of cars sold in market-year (ma, ye)
cars['qu_tot'] = cars.groupby(['ma', 'ye'])['qu'].transform('sum')
cars['market_size'] = cars['pop'] * cars['ad_w_li']
cars['s'] = cars['qu'] / cars['market_size']

In [10]:
# compute the share of the outside good (will be useful for the demand inversion)
cars['s0'] = 1.0 - cars.groupby(['ma', 'ye'])['s'].transform('sum')
print(f'Outside share is from {cars.s0.min():.1%} to {cars.s0.max():.1%}')

Outside share is from 93.1% to 97.1%


In [11]:
cars.groupby(['ma'])['s'].describe().rename(index=lbl_vals['market']).style.format('{:.3f}')

,count,unique,top,freq
ma,,,,
Belgium,1200.000,1194.000,0.003,2.000
France,1200.000,1199.000,0.001,2.000
Germany,1200.000,1199.000,0.000,2.000
Italy,1200.000,1195.000,0.000,2.000
UK,1200.000,1199.000,0.000,2.000


## 1. Using canned software

In [12]:
from linearmodels.iv import IV2SLS

In [13]:
cars['delta'] = cars['s'] / cars['s0']
cars['delta'] = np.log(cars['delta'].values.astype(float)) ## Den stoppede med at ville gøre det i et skridt uden at definere type

In [14]:
cars["brand"].replace('alfa romeo', 'alfa_romeo', inplace=True)
cars["brand"] = cars["brand"].str.replace('/', '', regex=False)

In [15]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

In [16]:
# Calculate the average price of all other cars in a given year in a given market:
# Step 1: Calculate the sum and count of prices for each year and market
cars['sum_eurpr_ye_ma'] = cars.groupby(['ye', 'ma'])['eurpr'].transform('sum')
cars['count_ye_ma'] = cars.groupby(['ye', 'ma'])['eurpr'].transform('count')

# Step 2: Calculate the average price excluding the current observation
cars['avg_eurpr_excl'] = (cars['sum_eurpr_ye_ma'] - cars['eurpr']) / (cars['count_ye_ma'] - 1)

# Drop the intermediate columns if they are no longer needed
cars.drop(columns=['sum_eurpr_ye_ma', 'count_ye_ma'], inplace=True)


cars['avg_eurpr_excl'] = np.log(cars['avg_eurpr_excl'])

In [17]:
# choose your preferred variables 
x_vars = ['logp', 'avg_eurpr_excl', 'home', 'cy', 'hp', 'we', 'li', 'sp'] + x_vars_dummies # <--- !!! CHOOSE HERE 
print(x_vars)

['logp', 'avg_eurpr_excl', 'home', 'cy', 'hp', 'we', 'li', 'sp', 'MCC', 'VW', 'alfa_romeo', 'audi', 'citroen', 'daewoo', 'daf', 'fiat', 'ford', 'honda', 'hyundai', 'innocenti', 'lancia', 'mazda', 'mercedes', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'renault', 'rover', 'saab', 'seat', 'skoda', 'suzuki', 'talbot', 'talhillman', 'talmatra', 'talsimca', 'talsunb', 'toyota', 'volvo']


In [18]:
# set up the estimation equation
formula = 'delta ~ 1'
for x_ in x_vars:
    formula += ' + ' + x_
print(formula)

delta ~ 1 + logp + avg_eurpr_excl + home + cy + hp + we + li + sp + MCC + VW + alfa_romeo + audi + citroen + daewoo + daf + fiat + ford + honda + hyundai + innocenti + lancia + mazda + mercedes + mitsubishi + nissan + opel + peugeot + renault + rover + saab + seat + skoda + suzuki + talbot + talhillman + talmatra + talsimca + talsunb + toyota + volvo


In [19]:
# Estimate the model by OLS
OLSmodel = IV2SLS.from_formula(formula, cars).fit()
OLSmodel.summary

c:\Users\PSand\anaconda3\lib\site-packages\linearmodels\iv\model.py:557: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:                  delta   R-squared:                      0.4050
Estimator:                        OLS   Adj. R-squared:                 0.4010
No. Observations:                5998   F-statistic:                 1.507e+05
Date:                Sat, Oct 19 2024   P-value (F-stat)                0.0000
Time:                        13:51:08   Distribution:                 chi2(40)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
==================================================================================
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
Intercept         -9.2503     0.2569    -36.008     0.0000     -9.7538     -8.7468
logp              -0.4723     0.0879    -5.3741     0.0000     -0.6446     -0.3001
avg_eurpr_excl     0.5152     0.0799     6.4519     0.0000      0.3587      0.6717
home               1.0296     0.0232     44.291     0.0000      0.9840      1.0751
cy                -0.0002  7.699e-05    -2.1237     0.0337     -0.0003  -1.261e-05
hp                -0.0285     0.0022    -12.964     0.0000     -0.0328     -0.0242
we                 0.0008     0.0001     5.1978     0.0000      0.0005      0.0010
li                -0.0128     0.0113    -1.1326     0.2574     -0.0350      0.0094
sp                 0.0179     0.0017     10.380     0.0000      0.0145      0.0212
MCC               -1.2479     0.1561    -7.9928     0.0000     -1.5539     -0.9419
VW                 0.0244     0.0580     0.4211     0.6737     -0.0893      0.1381
alfa_romeo        -0.5549     0.0653    -8.5019     0.0000     -0.6828     -0.4269
audi              -0.1432     0.0574    -2.4932     0.0127     -0.2557     -0.0306
citroen           -0.4319     0.0547    -7.8945     0.0000     -0.5391     -0.3247
daewoo            -0.7563     0.1571    -4.8135     0.0000     -1.0643     -0.4484
daf               -0.7957     0.1600    -4.9721     0.0000     -1.1093     -0.4820
fiat              -0.3091     0.0548    -5.6444     0.0000     -0.4164     -0.2017
ford               0.0701     0.0532     1.3182     0.1874     -0.0341      0.1744
honda             -0.3504     0.0752    -4.6603     0.0000     -0.4977     -0.2030
hyundai           -1.0150     0.1357    -7.4775     0.0000     -1.2810     -0.7489
innocenti         -1.3010     0.2282    -5.7019     0.0000     -1.7482     -0.8538
lancia            -0.8437     0.0747    -11.300     0.0000     -0.9901     -0.6974
mazda             -0.2917     0.0652    -4.4751     0.0000     -0.4195     -0.1640
mercedes           0.4302     0.0584     7.3640     0.0000      0.3157      0.5447
mitsubishi        -0.3679     0.0763    -4.8216     0.0000     -0.5174     -0.2183
nissan            -0.2879     0.0545    -5.2863     0.0000     -0.3947     -0.1812
opel              -0.0634     0.0566    -1.1213     0.2622     -0.1743      0.0474
peugeot           -0.1864     0.0566    -3.2909     0.0010     -0.2975     -0.0754
renault           -0.0808     0.0515    -1.5707     0.1163     -0.1817      0.0200
rover             -0.5230     0.0604    -8.6584     0.0000     -0.6414     -0.4046
saab              -0.9340     0.1555    -6.0063     0.0000     -1.2388     -0.6292
seat              -0.8033     0.0708    -11.348     0.0000     -0.9420     -0.6646
skoda             -1.1169     0.1072    -10.414     0.0000     -1.3271     -0.9067
suzuki            -0.9155     0.0895    -10.233     0.0000     -1.0908     -0.7401
talbot            -0.5084     0.0789    -6.4430     0.0000     -0.6630     -0.353

The Logit elasticities are 

$$
\mathcal{E}_{jk} \equiv \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = 
\begin{cases}
\alpha (\mathbf{1}\{j = k\} - s_{jt}) p_{kt} & \text{if price is in level},   \\
\alpha (\mathbf{1}\{j = k\} - s_{jt})        & \text{if price is in log }. 
\end{cases}
$$

In [20]:
beta = OLSmodel.params
elast_own = beta['logp'] * (1 - cars['s'])
print(f'Price in logs:  Avg. own-price elasticity: {elast_own.mean(): .2%}')

elast_cross = - beta['logp'] * cars['s']
print(f'Price in logs:  Avg. cross-price elasticity: {elast_cross.mean(): .2%}')

Price in logs:  Avg. own-price elasticity: -47.17%
Price in logs:  Avg. cross-price elasticity:  0.06%


$$
\frac{\partial s_{jt}}{\partial p_{kt}} = 
\begin{cases}
\alpha (\mathbf{1}\{j = k\} - s_{jt}) s_{jt} & \text{if price is in level},   \\
\alpha (\mathbf{1}\{j = k\} - s_{jt}) \frac{s_{jt}} {p_{kt}}      & \text{if price is in log }. 
\end{cases}
$$

In [21]:
cars['idx'] = cars.index

In [22]:
def MarginalCost(dat, index, beta, alpha, price, share, firm, log=True):
    p = dat[price].values
    firms = dat[firm].values
    H = (firms[:, None] == firms[None, :]).astype(np.int8)
    s = dat[share].values       # Column 's'
    alpha = beta[alpha]              # The given alpha value

    # Compute the size of the matrix
    n = len(dat)

    # Create an identity matrix for the I(j = k) term
    id_mat = np.eye(n)

    divisor = 1
    if log:
        divisor = p[:, None]

    # Compute s_dif matrix
    s_dif = (alpha / divisor) * s[:, None] * (id_mat - s)

    # Ensure H and s_dif are numeric arrays
    H = np.asarray(H, dtype=np.float64)
    s_dif = np.asarray(s_dif, dtype=np.float64)

    # Multiply H and s_dif element-wise
    Hs_dif = np.multiply(H, s_dif)

    # Ensure s_values is reshaped correctly and is a numeric array
    s = s.reshape((n, 1)).astype(np.float64)

    # Ensure p_values is also a numeric array
    p = np.asarray(p, dtype=np.float64)
    p = p.reshape((n, 1)).astype(np.float64)

    # Solve the system of equations hs_dif * c = s_values
    mc = p + np.linalg.inv(Hs_dif) @ s
    idx = dat[index].values
    idx = idx.reshape((n, 1)).astype(np.float64)
    return np.hstack((idx, mc))

In [23]:
def MC_loop(dat, index, beta, alpha, price, share, firm, market, year, log=True):

    mc_list = []

    for ye in dat[year].unique():
        for ma in dat[market].unique():
            sub_dat = dat[(dat[market] == ma) & (dat[year] == ye)].copy()
            mc = MarginalCost(sub_dat, index, beta, alpha, price, share, firm, log)
            mc_list.append(mc)

    # Vertically stack all arrays in the list into a single 2D array
    stacked_mc = np.vstack(mc_list)
    return stacked_mc

In [34]:
mc_array = MC_loop(cars, 'idx', beta, 'logp', 'eurpr', 's', 'frm', 'ma', 'ye', log=True)  # mc_array[:, 0] is idx values, mc_array[:, 1] is mc values
mc_dict = dict(mc_array)
cars['mc_vanilla'] = cars['idx'].map(mc_dict)

In [ ]:
def ccp(p, t): 
    '''
    INPUTS: 
        p: (J,) vector of prices
        t: (int) market index
    OUTPUTS:
        ccp: (J+1,) vector of conditional choice probabilities (0 = outside option)
    '''
    assert p.shape == (J,)

    # 1. compute utilities for all J products 
    delta = x1[t,:]*beta[0] + x2[t,:]*beta[1] + alpha*p + xi[t,:] # (J,)
    
    # 2. insert a zero in the first position for the outside option
    delta = np.insert(delta, 0, 0.0)
    
    # 3. max-rescale (to avoid numerical issues)
    delta -= delta.max() # no need for keepdims=True since delta.max() is a scalar

    # 4. compute the CCP
    ed = np.exp(delta)
    ccp = ed / ed.sum()

    return ccp # (J+1,)

In [ ]:
def zeta(p, mc, H, t): 
    assert (p.shape == (J,)) and (mc.shape == (J,)) and (H.shape == (J, J))
    s = ccp(p, t)
    s = s[1:] # remove outside option 

    Lambda = alpha * np.diag(s) * H 
    Gamma = alpha * np.outer(s, s) * H
    z = np.linalg.inv(Lambda) @ (Gamma @ (p - mc) - s)
    return z

In [ ]:
def solve_nash_MS(p_start, mc, H, t, maxit=1000, tol=1e-6, DOPRINT=False): 
    p_prev = p_start.copy()
    for it in range(maxit): 
        p_next = mc + zeta(p_prev, mc, H, t)
        if np.linalg.norm(p_next - p_prev) < tol: 
            if DOPRINT: 
                print(f'Converged after {it} iterations')
            break 
        p_prev = p_next
    return p_next

In [ ]:
p_start = mc[t,:]*1.5
H = np.eye(J)
p_NE_Z = solve_nash_MS(p_start, mc[t,:], H, t)
p_NE_Z

## IV-estimation

We use the Hausman-instrument. Start by computing this for the M-1 markets for each car.

In [26]:
# Ensure that the the country-column is treated as a string in the dataframe
cars['ma'] = cars['ma'].astype(str)

# Get the unique list of countries as strings
countries = cars['ma'].unique()

# Step 2: Create the four additional columns for prices from the other four countries
for country in countries:
    # Create a new column for each country
    column_name = f'eurpr_in_{country}'
    
    # Create a copy of the dataframe that only includes the rows for the given country
    country_data = cars[cars['ma'] == country][['ye', 'type', 'eurpr']]
    
    # Rename the 'eurpr' column in this temporary dataframe to avoid confusion
    country_data = country_data.rename(columns={'eurpr': column_name})
    
    # Merge the country-specific prices back into the main dataframe on 'ye' and 'type'
    cars = cars.merge(country_data, on=['ye', 'type'], how='left')

# Step 4: For each row, remove the price of the current country from the new columns
for country in countries:
    column_name = f'eurpr_in_{country}'
    
    # Set the column to NaN where the country matches the current row
    cars.loc[cars['ma'] == country, column_name] = None

In [27]:
price_columns = [f'eurpr_in_{country}' for country in countries]

In [28]:
# this is the Hausman-instrument we apply
cars['avg_eurpr_other'] = cars[price_columns].mean(axis=1)


In [29]:
# Estimate the model by OLS
IVmodel = IV2SLS.from_formula(formula, cars).fit()
OLSmodel.summary

c:\Users\PSand\anaconda3\lib\site-packages\linearmodels\iv\model.py:557: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:                  delta   R-squared:                      0.4050
Estimator:                        OLS   Adj. R-squared:                 0.4010
No. Observations:                5998   F-statistic:                 1.507e+05
Date:                Sat, Oct 19 2024   P-value (F-stat)                0.0000
Time:                        13:51:08   Distribution:                 chi2(40)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
==================================================================================
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
Intercept         -9.2503     0.2569    -36.008     0.0000     -9.7538     -8.7468
logp              -0.4723     0.0879    -5.3741     0.0000     -0.6446     -0.3001
avg_eurpr_excl     0.5152     0.0799     6.4519     0.0000      0.3587      0.6717
home               1.0296     0.0232     44.291     0.0000      0.9840      1.0751
cy                -0.0002  7.699e-05    -2.1237     0.0337     -0.0003  -1.261e-05
hp                -0.0285     0.0022    -12.964     0.0000     -0.0328     -0.0242
we                 0.0008     0.0001     5.1978     0.0000      0.0005      0.0010
li                -0.0128     0.0113    -1.1326     0.2574     -0.0350      0.0094
sp                 0.0179     0.0017     10.380     0.0000      0.0145      0.0212
MCC               -1.2479     0.1561    -7.9928     0.0000     -1.5539     -0.9419
VW                 0.0244     0.0580     0.4211     0.6737     -0.0893      0.1381
alfa_romeo        -0.5549     0.0653    -8.5019     0.0000     -0.6828     -0.4269
audi              -0.1432     0.0574    -2.4932     0.0127     -0.2557     -0.0306
citroen           -0.4319     0.0547    -7.8945     0.0000     -0.5391     -0.3247
daewoo            -0.7563     0.1571    -4.8135     0.0000     -1.0643     -0.4484
daf               -0.7957     0.1600    -4.9721     0.0000     -1.1093     -0.4820
fiat              -0.3091     0.0548    -5.6444     0.0000     -0.4164     -0.2017
ford               0.0701     0.0532     1.3182     0.1874     -0.0341      0.1744
honda             -0.3504     0.0752    -4.6603     0.0000     -0.4977     -0.2030
hyundai           -1.0150     0.1357    -7.4775     0.0000     -1.2810     -0.7489
innocenti         -1.3010     0.2282    -5.7019     0.0000     -1.7482     -0.8538
lancia            -0.8437     0.0747    -11.300     0.0000     -0.9901     -0.6974
mazda             -0.2917     0.0652    -4.4751     0.0000     -0.4195     -0.1640
mercedes           0.4302     0.0584     7.3640     0.0000      0.3157      0.5447
mitsubishi        -0.3679     0.0763    -4.8216     0.0000     -0.5174     -0.2183
nissan            -0.2879     0.0545    -5.2863     0.0000     -0.3947     -0.1812
opel              -0.0634     0.0566    -1.1213     0.2622     -0.1743      0.0474
peugeot           -0.1864     0.0566    -3.2909     0.0010     -0.2975     -0.0754
renault           -0.0808     0.0515    -1.5707     0.1163     -0.1817      0.0200
rover             -0.5230     0.0604    -8.6584     0.0000     -0.6414     -0.4046
saab              -0.9340     0.1555    -6.0063     0.0000     -1.2388     -0.6292
seat              -0.8033     0.0708    -11.348     0.0000     -0.9420     -0.6646
skoda             -1.1169     0.1072    -10.414     0.0000     -1.3271     -0.9067
suzuki            -0.9155     0.0895    -10.233     0.0000     -1.0908     -0.7401
talbot            -0.5084     0.0789    -6.4430     0.0000     -0.6630     -0.353